In [18]:
#Import the libraries
import os, random
import numpy as np
import pandas as pd
import tensorflow as tf
import librosa as lib
import librosa.display
import matplotlib.pyplot as plt
import seaborn as sb
from scipy.io import wavfile as wav
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.metrics import accuracy_score

from tensorflow.keras.layers import Dense,Dropout, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.models import Sequential
import IPython.display as ipd
%matplotlib inline

In [20]:
#Access the audio image filepath
train_audio_path = '../Dataset audio samples/classification dataset/'
test_audio_path = '../Dataset audio samples/classification test samples/'
labels = os.listdir(train_audio_path)
print(labels)

['left', 'one', 'right', 'stop', 'three', 'two']


In [3]:
#Preprocessing below:
#Resampling at 8000kHz
#Filtering out audio <1s in length
#Creating arrays of audio and its labels an dcreate a dataframe from them

x_audio_array=[]
y_label_array=[]
sample_rate = 8000
count = 0
count1 = 0

for label in labels:
    audios = [f for f in os.listdir(train_audio_path+label) if f.endswith('.wav')]
    for aud in audios:
        #Resampling
        sam,rate = lib.load(train_audio_path+label+'/'+aud, sr=sample_rate)
        if len(sam)/rate == 1:
            x_audio_array.append(sam)
            y_label_array.append(label)

In [4]:
#Encoding categorical data(labels)
lb = LabelBinarizer()
y_encoded = lb.fit_transform(y_label_array)
print(y_encoded.shape)

(12837, 6)


In [5]:
x_audios = np.array(x_audio_array).reshape(-1,8000,1)
print(x_audios.shape)

(12837, 8000, 1)


In [6]:
#Split datasets into training and test data
x_train, x_test, y_train, y_test = train_test_split(x_audios, y_encoded, test_size=0.3, random_state=0)

In [7]:
print(x_train.shape)
print(y_train.shape)

(8985, 8000, 1)
(8985, 6)


In [13]:
#Create 1D conv1d CNN model

cnn1d = Sequential()

#1st 1D conv layer and maxpooling layer
cnn1d.add(Conv1D(filters=16, kernel_size = 8, activation='relu', input_shape=(8000,1)))
cnn1d.add(MaxPooling1D(pool_size=2, strides=2))

#2nd 1D conv layer and maxpooling layer
cnn1d.add(Conv1D(filters=16, kernel_size = 8, activation='relu'))
cnn1d.add(MaxPooling1D(pool_size=2, strides=2))
cnn1d.add(Dropout(0.3))

#3rd 1D conv layer and maxpooling layer
cnn1d.add(Conv1D(filters=16, kernel_size = 8, activation='relu'))
cnn1d.add(MaxPooling1D(pool_size=2, strides=2))
cnn1d.add(Dropout(0.3))

#4th 1D conv layer and maxpooling layer
cnn1d.add(Conv1D(filters=16, kernel_size = 8, activation='relu'))
cnn1d.add(MaxPooling1D(pool_size=2, strides=2))
cnn1d.add(Dropout(0.3))

#Flattening
cnn1d.add(Flatten())

#Fully connected layer
cnn1d.add(Dense(units=128, activation='relu'))
cnn1d.add(Dropout(0.3))

#Output layer
cnn1d.add(Dense(units=6, activation='softmax'))

cnn1d.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 7993, 16)          144       
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 3996, 16)          0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 3989, 16)          2064      
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 1994, 16)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1994, 16)          0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 1987, 16)          2064      
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 993, 16)          

In [14]:
#Compile CNN
cnn1d.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#Fit the model(train)
cnn1d.fit(x_train, y_train, batch_size=32, epochs=150)

Epoch 1/150
281/281 [==============================] - 69s 247ms/step - loss: 1.5792 - accuracy: 0.3006
Epoch 2/150
281/281 [==============================] - 69s 245ms/step - loss: 1.1340 - accuracy: 0.5308
Epoch 3/150
281/281 [==============================] - 69s 244ms/step - loss: 0.9857 - accuracy: 0.5994
Epoch 4/150
281/281 [==============================] - 70s 247ms/step - loss: 0.8627 - accuracy: 0.6472
Epoch 5/150
281/281 [==============================] - 71s 254ms/step - loss: 0.7911 - accuracy: 0.6874
Epoch 6/150
281/281 [==============================] - 75s 266ms/step - loss: 0.7231 - accuracy: 0.7062
Epoch 7/150
281/281 [==============================] - 75s 266ms/step - loss: 0.6504 - accuracy: 0.7344
Epoch 8/150
281/281 [==============================] - 75s 268ms/step - loss: 0.5980 - accuracy: 0.7600
Epoch 9/150
281/281 [==============================] - 75s 265ms/step - loss: 0.5563 - accuracy: 0.7784
Epoch 10/150
281/281 [==============================] - 72s 257m

281/281 [==============================] - 71s 254ms/step - loss: 0.0957 - accuracy: 0.9701
Epoch 75/150
281/281 [==============================] - 70s 248ms/step - loss: 0.0845 - accuracy: 0.9721
Epoch 76/150
281/281 [==============================] - 63s 224ms/step - loss: 0.0915 - accuracy: 0.9685
Epoch 77/150
281/281 [==============================] - 62s 222ms/step - loss: 0.0861 - accuracy: 0.9720
Epoch 78/150
281/281 [==============================] - 61s 218ms/step - loss: 0.0864 - accuracy: 0.9716
Epoch 79/150
281/281 [==============================] - 60s 214ms/step - loss: 0.0803 - accuracy: 0.9731
Epoch 80/150
281/281 [==============================] - 62s 219ms/step - loss: 0.0879 - accuracy: 0.9684
Epoch 81/150
281/281 [==============================] - 61s 218ms/step - loss: 0.0816 - accuracy: 0.9723
Epoch 82/150
281/281 [==============================] - 57s 204ms/step - loss: 0.0900 - accuracy: 0.9703
Epoch 83/150
281/281 [==============================] - 57s 201ms/st

281/281 [==============================] - 54s 192ms/step - loss: 0.0648 - accuracy: 0.9785
Epoch 144/150
281/281 [==============================] - 56s 200ms/step - loss: 0.0712 - accuracy: 0.9750
Epoch 145/150
281/281 [==============================] - 56s 200ms/step - loss: 0.0689 - accuracy: 0.9771
Epoch 146/150
281/281 [==============================] - 56s 200ms/step - loss: 0.0657 - accuracy: 0.9769
Epoch 147/150
281/281 [==============================] - 56s 199ms/step - loss: 0.0639 - accuracy: 0.9758
Epoch 148/150
281/281 [==============================] - 60s 212ms/step - loss: 0.0752 - accuracy: 0.9748
Epoch 149/150
281/281 [==============================] - 57s 202ms/step - loss: 0.0579 - accuracy: 0.9804s - loss: 0
Epoch 150/150
281/281 [==============================] - 56s 199ms/step - loss: 0.0672 - accuracy: 0.9790


In [51]:
# Evaluating the model on the training and testing set
score = cnn1d.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = cnn1d.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.9939900040626526
Testing Accuracy:  0.8582554459571838


In [52]:
#Predict using ann model
res=cnn1d.predict(x_test)
print(lb.inverse_transform((res[0].reshape(1,-1)))[0])
print('Accuracy score: ', accuracy_score(lb.inverse_transform(y_test),lb.inverse_transform(res)))

stop
Accuracy score:  0.8582554517133957


In [53]:
import sounddevice as sd
from scipy.io.wavfile import write

def record_audio():
    rate = 8000  # Sample rate
    seconds = 1  # Duration of recording

    my_audio = sd.rec(int(seconds * rate), samplerate=rate, channels=1)
    sd.wait()  # Wait until recording is finished
    write(test_audio_path + 'test.wav', rate, my_audio)  # Save as WAV file 
    return 'test.wav'
    
def predict_audio(filename):
    filepath = test_audio_path + filename
    audio, sam = lib.load(filepath, sr=8000)
    pad_width = 8000 - audio.shape[0]
    audio = np.pad(audio, pad_width=(0,pad_width))
    audio_predicted=cnn1d.predict(audio.reshape(1,audio.shape[0],1))
    audio_class = lb.inverse_transform(audio_predicted) 
    print('Your audio (' + filename + ') belongs to the label: ', audio_class[0].upper())

In [54]:
#Classify a random audio file from the unlabeled test folder
file = random.choice([f for f in os.listdir(test_audio_path)])
predict_audio(file)
ipd.Audio(test_audio_path+file, autoplay=True)

Your audio (stop (5).wav) belongs to the label:  STOP


In [63]:
#Classify a live spoken audio through microphone
predict_audio(record_audio())
ipd.Audio(test_audio_path+'test.wav', autoplay=True)

Your audio (test.wav) belongs to the label:  THREE
